### Tworzenie REST API

Na potrzeby testów do pliku schema.sql wpisujemy

In [ ]:
CREATE TABLE post (
    id BIGINT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(40) NOT NULL,
    content VARCHAR(40) NOT NULL,
    created TIMESTAMP
);

CREATE TABLE comment (
    id BIGINT AUTO_INCREMENT PRIMARY KEY,
    post_id BIGINT NOT NULL,
    content VARCHAR(2000) NULL,
    created TIMESTAMP
);

ALTER TABLE comment
    ADD CONSTRAINT comment_post_id
    FOREIGN KEY (post_id) REFERENCES post(id);

A w pliku build.gradle dodajemy następyjący kod i uruchamiamy ten plik!
Dzięki temu zapiszemy w pliku data.sql kod, który doda testowe wartości do bazy danych.

In [ ]:
task generatePostsAndComments() {
    doLast {
    File dataSql = file("src/main/resources/data.sql")
        dataSql.write("")
    // posts
    for (int i = 1; i < 100; i++) {
        dataSql.append("insert into post(id, title, content, created) " +
            "values (${i}, 'Test post ${i}', 'Content ${i}', '" + LocalDateTime.now().minusDays(100 - i) + "');\n")
    }

    //comments
    for (int i = 1; i < 100; i++ ) {
        int postId = 1 + i / 10;
        dataSql.append("insert into comment(id, post_id, content, created) " +
            "values (${i}, ${postId}, 'Comment ${i}', '" + LocalDateTime.now().minusDays(100 - i) + "');\n")
        }
    }
}

Tworzymy nowy pakiet z interfejsem ../repository.PostRepository

In [ ]:
import org.springframework.data.jpa.repository.JpaRepository;
import org.springframework.stereotype.Repository;
import pl.piotrfolio.restapi.model.Post;

@Repository
public interface PostRepository extends JpaRepository<Post, Long> {
}

Tworzymy plik PostController w utworzonym wcześniej pakiecie .../controller/

In [ ]:
@RestController
public class PostController {

    @GetMapping("/posts")
    public List<Post> getPosts() {
        return postService.getPosts();
    }

    @GetMapping("/posts/{id}")
    public Post getSinglePost(@PathVariable long id) {  ## long id z modelu Post
        return postService.getSinglePost(id);;
    }
}

Tworzymy nowy pakiet z klasą "model.Post", który ma takie same pola (zmienne) jak tabela w bazie danych.
Tworzymy również gettery i settery.

In [ ]:
@Entity
public class Post {

    @Id  ## tutaj musi być import jakarta.persistence.Id; inaczej nie będzie działać!!
    private long id;
    
    private String title;
    private String content;
    private LocalDateTime created;

    public long getId() {
        return id;
    }

    public String getTitle() {
        return title;
    }

    public String getContent() {
        return content;
    }

    public LocalDateTime getCreated() {
        return created;
    }

    public void setId(long id) {
        this.id = id;
    }

    public void setTitle(String title) {
        this.title = title;
    }

    public void setContent(String content) {
        this.content = content;
    }

    public void setCreated(LocalDateTime created) {
        this.created = created;
    }
}

Tworzymy nowy serwis PostService w ../service. Serwis pozwala na implementowanie CRUD-owych metod do komunikowania się z bazą danych.

In [ ]:
import org.springframework.stereotype.Service;
import pl.piotrfolio.restapi.model.Post;
import pl.piotrfolio.restapi.repository.PostRepository;

import java.util.List;

@Service
//@RequiredArgsConstructor
public class PostService {

    private final PostRepository postRepository;

    public PostService(PostRepository postRepository) {
        this.postRepository = postRepository;
    }

    public List<Post> getPosts() {
        return postRepository.findAll();
    }

    public Post getSinglePost(long id) {
        return postRepository.findById(id).orElseThrow();
    }
}

Tworzymy nową encję (model) dla sekcji Comment

In [ ]:
@Entity
@Getter  ## annotation from Lombok
@Setter  ## annotation from Lombok
public class Comment {

    @Id
    private long id;
    private String content;
    private LocalDateTime created;
}

## there is no variable for foreign key field, it's not necessary because it will be "implemented" in Post model
## it's not necessary because it only represents the model to display when fetched from API. 

W modelu Post dopisujemy 

In [ ]:
@Id
    private long id;

    private String title;
    private String content;
    private LocalDateTime created;

    ## we have to add annotation 
    @OneToMany
    @JoinColumn(name = "post_id") # name of the column with a foreign key in Comment model.
    ## in the later stage, if we want to use it
    private List<Comment> comment;